In [1]:
from collections import deque
from model import RL_GRU2
from dataUtils import *
from logger import MyLogger
import sys

W0813 16:10:10.932252 139974489069376 deprecation_wrapper.py:119] From /home/hadoop/ERD/model.py:3: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

Using TensorFlow backend.
W0813 16:10:10.975089 139974489069376 deprecation_wrapper.py:119] From /home/hadoop/.conda/envs/TF/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0813 16:10:10.975697 139974489069376 deprecation_wrapper.py:119] From /home/hadoop/.conda/envs/TF/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 16:10:10.977313 139974489069376 deprecation_wrapper.py:119] From /home/hadoop/.conda/envs/TF/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0813 16:10:

In [1]:
import lstm_char_cnn
import tensorflow as tf

tf.app.flags.DEFINE_string('f', '', 'kernel')

import PTB_data_reader
import config
FLAGS = tf.flags.FLAGS
word_vocab, char_vocab, word_tensors, char_tensors, max_word_length = \
PTB_data_reader.load_data(FLAGS.data_dir, FLAGS.max_word_length, eos=FLAGS.EOS)

train_reader =PTB_data_reader.DataReader(word_tensors['train'], char_tensors['train'],
                              FLAGS.batch_size, FLAGS.num_unroll_steps)

reading train
reading valid
reading test

actual longest token length is: 21
size of word vocabulary: 10000
size of char vocabulary: 51
number of tokens in train: 929589
number of tokens in valid: 73760
number of tokens in test: 82430


In [2]:
input_train = tf.placeholder(tf.int32, shape=[FLAGS.batch_size, FLAGS.num_unroll_steps, max_word_length], name="input")
char_model = lstm_char_cnn.inference_graph(
                    char_vocab_size=char_vocab.size,
                    word_vocab_size=word_vocab.size,
                    input_ = input_train,
                    char_embed_size=FLAGS.char_embed_size,
                    batch_size=FLAGS.batch_size,
                    num_highway_layers=FLAGS.highway_layers,
                    num_rnn_layers=FLAGS.rnn_layers,
                    rnn_size=FLAGS.rnn_size,
                    max_word_length=50,
                    kernels=eval(FLAGS.kernels),
                    kernel_features=eval(FLAGS.kernel_features),
                    num_unroll_steps=FLAGS.num_unroll_steps,
                    dropout=FLAGS.dropout
    )
char_model.update(lstm_char_cnn.loss_graph(char_model.logits, FLAGS.batch_size, FLAGS.num_unroll_steps))
char_model.update(lstm_char_cnn.training_graph(char_model.loss * FLAGS.num_unroll_steps,
                FLAGS.learning_rate, FLAGS.max_grad_norm))

W0813 21:03:58.122577 140435558864704 deprecation_wrapper.py:119] From /home/hadoop/ERD/lstm_char_cnn.py:129: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0813 21:03:58.123331 140435558864704 deprecation_wrapper.py:119] From /home/hadoop/ERD/lstm_char_cnn.py:130: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0813 21:03:58.124018 140435558864704 deprecation.py:506] From /home/hadoop/.conda/envs/TF/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0813 21:03:58.131158 140435558864704 deprecation_wrapper.py:119] From /home/hadoop/ERD/lstm_char_cnn.py:137: The name tf.scatter_update is deprecated. Please use tf.compat.v1.scatter_upda

In [3]:
saver = tf.train.Saver(tf.global_variables(), max_to_keep=4)
session = tf.Session()
with session.as_default():
    session.run(tf.global_variables_initializer())
summary_writer = tf.summary.FileWriter(FLAGS.train_dir, graph=session.graph)

In [8]:
import time
import numpy as np
def Train_Char_Model(session, input_train, train_model, train_reader, saver, summary_writer):
    best_valid_loss = None
    rnn_state = session.run(train_model.initial_rnn_state)
#     for epoch in range(FLAGS.max_epochs):
    for epoch in range(1):
        epoch_start_time = time.time()
        avg_train_loss = 0.0
        count = 0
        for x, y in train_reader.iter():
            count += 1
            start_time = time.time()

            loss, _, rnn_state, gradient_norm, step, _ = session.run([
                train_model.loss,
                train_model.train_op,
                train_model.final_rnn_state,
                train_model.global_norm,
                train_model.global_step,
                train_model.clear_char_embedding_padding
            ], {
                input_train: x,
                train_model.targets: y,
                train_model.initial_rnn_state: rnn_state
            })

            summary = tf.Summary(value=[
                tf.Summary.Value(tag="step_train_loss", simple_value=loss),
                tf.Summary.Value(tag="step_train_perplexity", simple_value=np.exp(loss)),
            ])
            summary_writer.add_summary(summary, step)

            avg_train_loss += 0.05 * (loss - avg_train_loss)

            time_elapsed = time.time() - start_time

            if count % FLAGS.print_every == 0:
                print('%6d: %d [%5d/%5d], train_loss/perplexity = %6.8f/%6.7f secs/batch = %.4fs, grad.norm=%6.8f' % (step,
                                                        epoch, count,
                                                        train_reader.length,
                                                        loss, np.exp(loss),
                                                        time_elapsed,
                                                        gradient_norm))
                break
        print('Epoch training time:', time.time()-epoch_start_time)
        save_as = '%s/epoch%03d_%.4f.model' % (FLAGS.train_dir, epoch, avg_train_loss)
        saver.save(session, save_as)
        print('Saved char model', save_as)

In [9]:
Train_Char_Model(session, input_train, char_model, train_reader, saver, summary_writer)

    11: 0 [    5/  531], train_loss/perplexity = 7.78428316/2402.5432129 secs/batch = 2.2163s, grad.norm=7.60929680
Epoch training time: 10.185387372970581
Saved char model cv/epoch000_1.7867.model


In [11]:
rnn_state = session.run(char_model.initial_rnn_state)

In [17]:
x_in = np.random.randint(0, char_vocab.size, size = [FLAGS.batch_size, FLAGS.num_unroll_steps, max_word_length])

In [21]:
cnn_o= session.run([
                char_model.input_cnn
            ], {
                input_train: x_in,
                char_model.initial_rnn_state: rnn_state
            })


In [23]:
cnn_o[0].shape

(50, 35, 462)